# Replicating The pipeline() Function Functionality

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "It is really hot here in Italy and going outside means being drenched in your own sweat and it does not feel good.",
        "The weather in my hometown during summer is really nice!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9996693134307861},
 {'label': 'POSITIVE', 'score': 0.9998784065246582}]

When we run the above cell, we obtain
```
[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]
```
Now under the hood, the pipeline function is conducting all the tasks neccessary to use the specified model (in this case it is the default model, which is *distilbert-base-uncased-finetuned-sst-2-english*) for the prediction. The steps are, *preprocessing (tokenisation)*, *then passing that input to the model*, and finally, *postprocessing*:
<br />
<br />
![Pipeline Steps](data/chapter_2/pipeline_steps.png "The Three Steps!")